In [1]:
# import dependancies

import pandas as pd
import yfinance as yf
import datetime as dt
from datetime import date
import os
import json
import csv

# Define file paths
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
ticker_dir = os.path.join(parent_dir,'stock_scrape_load')
ticker_path = f"{ticker_dir}/top_100_company_tickers.csv"
output_path = os.path.join(parent_dir,'05_michael_test')

In [2]:
goog = yf.Ticker('GOOG')
goog.history(start = '2023-07-11', end = '2023-07-15')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-07-11 00:00:00-04:00,116.760002,118.224998,115.830002,117.709999,18286600,0.0,0.0
2023-07-12 00:00:00-04:00,119.300003,120.959999,119.000000,119.620003,22059600,0.0,0.0
2023-07-13 00:00:00-04:00,121.540001,125.334999,121.059998,124.830002,31535900,0.0,0.0
2023-07-14 00:00:00-04:00,125.129997,127.089996,124.900002,125.699997,20482800,0.0,0.0


In [3]:

# dates = ['2023-07-11','2023-07-12','2023-7-13','2023-07-14' ]

# Define function to grab requested into from yfinance
# This is a modified version of the function from the company_address_pull_nb.ipynb 
def get_info(ticker):
    print("processing... ", ticker)
    company = yf.Ticker(ticker)
    short_name = company.info['shortName']
    long_name = company.info['longName']
    exchange = company.info['exchange']
    prev_close = company.info['previousClose']
    data = {ticker:{
        'shortName':short_name,
        'longName':long_name,
        'exchange':exchange,
        'previousClose':prev_close
    }}
    return(data)

In [4]:
company_info = {}

with open(ticker_path, 'r') as myfile:
    reader = csv.DictReader(myfile)
    for row in reader:
        ticker = row['Ticker']
        try:
            row_data = get_info(ticker)
            company_info.update(row_data)
        except Exception as e:
            print("error processing ", ticker + ":", e)

processing...  AAPL
processing...  MSFT
processing...  GOOG
processing...  AMZN
processing...  NVDA
processing...  TSLA
processing...  BRK-B
processing...  META
processing...  V
processing...  UNH
processing...  LLY
processing...  WMT
processing...  JPM
processing...  JNJ
processing...  XOM
processing...  MA
processing...  PG
processing...  AVGO
processing...  ORCL
processing...  HD
processing...  MRK
processing...  CVX
processing...  KO
processing...  PEP
processing...  ABBV
processing...  COST
processing...  BAC
processing...  ADBE
processing...  MCD
processing...  CSCO
processing...  CRM
processing...  PFE
processing...  TMO
processing...  NFLX
processing...  ABT
processing...  AMD
processing...  DHR
processing...  CMCSA
processing...  TMUS
processing...  NKE
processing...  DIS
processing...  WFC
processing...  TXN
processing...  VZ
processing...  UPS
processing...  PM
processing...  NEE
processing...  RTX
processing...  MS
processing...  INTC
processing...  HON
processing...  BMY
p

In [5]:
fieldnames = ["ticker", "shortName", "longName", "exchange", "previousClose"]

output_filename = "company_info.csv"
output_filepath = f"{output_path}{output_filename}"

with open(output_filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for ticker, data in company_info.items():
        writer.writerow({"ticker": ticker,
                         "shortName": data.get("shortName", ""),
                         "longName": data.get("longName", ""),
                         "exchange": data.get("exchange", ""),
                         "previousClose": data.get("previousClose", "")})

print("Company data saved as", output_filename)

Company data saved as company_info.csv


In [23]:
test = yf.download('GOOG', start = '2023-07-13')
test.columns

[*********************100%***********************]  1 of 1 completed


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [9]:
ticker_df = pd.read_csv(ticker_path)

In [35]:
start = "2023-07-01"

history_df = pd.DataFrame()  # Initialize an empty dataframe to store the results

for ticker in ticker_df['Ticker']:
    print(f"Downloading {ticker} hisorical prices over requested range.")
    data = yf.download(ticker, start)
    data['ticker'] = ticker  # Add a new column with the ticker symbol
    history_df = history_df.append(data)

# Convert the dataframe to the desired JSON format
json_data = history_df.reset_index().groupby('ticker').apply(
    lambda x: x[['Open', 'High', 'Low', 'Close','Volume']].to_dict('records')
).reset_index().rename(columns={0: 'daily'}).to_json(orient='records')

# Save the JSON data to a file
with open(f"{output_path}/stock_data.json", 'w') as file:
    file.write(json_data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/var/folders/y9/z_8b8_4101z89mf3t6r1tf6m0000gn/T/ipykernel_62252/1249129438.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history_df = history_df.append(data)
/var/folders/y9/z_8b8_4101z89mf3t6r1tf6m0000gn/T/ipykernel_62252/1249129438.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history_df = history_df.append(data)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/var/folders/y9/z_8b8_4101z89mf3t6r1tf6m0000gn/T/ipykernel_62252/1249129438.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history_df = history_df.append(data)
/var/folders/y9/z_8b8_4101z89mf3t6r1tf6m0000gn/T/ipykernel_62252/1249129438.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history_df = history_df.append(data)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

/var/folders/y9/z_8b8_4101z89mf3t6r1tf6m0000gn/T/ipykernel_62252/1249129438.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history_df = history_df.append(data)
/var/folders/y9/z_8b8_4101z89mf3t6r1tf6m0000gn/T/ipykernel_62252/1249129438.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history_df = history_df.append(data)


[*********************100%***********************]  1 of 1 completed


/var/folders/y9/z_8b8_4101z89mf3t6r1tf6m0000gn/T/ipykernel_62252/1249129438.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history_df = history_df.append(data)


[*********************100%***********************]  1 of 1 completed


/var/folders/y9/z_8b8_4101z89mf3t6r1tf6m0000gn/T/ipykernel_62252/1249129438.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history_df = history_df.append(data)


In [37]:
history_df.reset_index(inplace=True)
history_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2023-07-03,193.779999,193.880005,191.759995,192.460007,192.460007,31458200,AAPL
1,2023-07-05,191.570007,192.979996,190.619995,191.330002,191.330002,46920300,AAPL
2,2023-07-06,189.839996,192.020004,189.199997,191.809998,191.809998,45094300,AAPL
3,2023-07-07,191.410004,192.669998,190.240005,190.679993,190.679993,46778000,AAPL
4,2023-07-10,189.259995,189.990005,187.039993,188.610001,188.610001,59922200,AAPL


In [38]:
history_df.to_json(f"{output_path}/stock_data2.json", orient = 'records')